# Experiment Notebook: {Descriptive Experiment Keywords}

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [18]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

time: 77.8 ms (started: 2023-01-30 17:58:34 +08:00)


In [19]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

time: 16.2 ms (started: 2023-01-30 17:58:34 +08:00)


In [20]:
# Import experiment templates
import experiments.templates.polygon.yield_analysis as cumulative_yield_analysis

time: 14.5 ms (started: 2023-01-30 17:58:34 +08:00)


In [21]:
# Inspect experiment template
display_code(cumulative_yield_analysis)

"""
# Cumulative Yield Analysis
"""

import copy

import model.constants as constants
from model.stochastic_processes import create_stochastic_process_realizations
from model.types import Stage
from experiments.default_experiment import experiment


# Make a copy of the default experiment to avoid mutation
experiment = copy.deepcopy(experiment)

DELTA_TIME = 7*constants.epochs_per_day  # epochs per timestep
SIMULATION_TIME_MONTHS = 12 * 10  # number of months
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME

# Generate stochastic process realizations
polygn_price_samples = create_stochastic_process_realizations("polygn_price_samples", timesteps=TIMESTEPS, dt=DELTA_TIME)

PoS_checkpoint_gas_cost = constants.PoS_checkpoint_gas_cost
parameter_overrides = {
    "polygn_price_process": [lambda run, timestep: polygn_price_samples[run - 1][timestep]],
    "Adoption_speed": [20],
    "checkpoint_gas_cost": [0.1 * PoS_checkpoint_gas_cost]
}

# Override default experiment Simulation and System Parameters related to timing
experiment.simulations[0].timesteps = TIMESTEPS
experiment.simulations[0].model.params.update({"dt": [DELTA_TIME]})

# Override default experiment System Parameters
experiment.simulations[0].model.params.update(parameter_overrides)

time: 40.7 ms (started: 2023-01-30 17:58:34 +08:00)


In [22]:
# gas cost comsumption and validator yield analysis
simulation_2b = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])

time: 15.3 ms (started: 2023-01-30 17:58:34 +08:00)


In [23]:
simulation_2b.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 5,
    "PRIVATE_CHAINS_CNT": 5,
})

PoS_checkpoint_gas_cost = simulation_2b.model.params['checkpoint_gas_cost'][0]
simulation_2b.model.params.update({
    'staking_mode': [
       "MultiStaking",
       "SingleStaking"
    ],
    
})

time: 13.8 ms (started: 2023-01-30 17:58:34 +08:00)


In [24]:
df_2b, _exceptions = run(simulation_2b)

2023-01-30 17:58:34,778 - root - INFO - Running experiment
2023-01-30 17:58:34,780 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-30 17:58:38,237 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-30 17:58:41,627 - root - INFO - Experiment complete in 6.848012924194336 seconds
2023-01-30 17:58:41,630 - root - INFO - Post-processing results
2023-01-30 17:58:41,757 - root - INFO - Post-processing complete in 0.13038229942321777 seconds
time: 7 s (started: 2023-01-30 17:58:34 +08:00)


In [36]:
visualizations.plot_yields_per_staking_mode(
    df_2b,
    subplot_titles=['Multi Staking', 'Single Staking']
)

time: 249 ms (started: 2023-01-30 17:59:53 +08:00)


In [26]:
visualizations.plot_yields_per_subset(df_2b, scenario_names={0: 'Multi Staking', 1: 'Single Staking'})

time: 65.5 ms (started: 2023-01-30 17:58:42 +08:00)


In [27]:
#df_2b[["total_profit","custom_hardware_costs","custom_costs","validator_checkpoint_costs","total_priority_fee_to_validators_polygn"]]

time: 12.7 ms (started: 2023-01-30 17:58:42 +08:00)


In [28]:
visualizations.plot_treasury_per_subset(
    df_2b,
    scenario_names=['Multi Staking', 'Single Staking']
)

time: 43.2 ms (started: 2023-01-30 17:58:42 +08:00)


In [29]:
visualizations.plot_token_price(
    df_2b,
)

time: 42 ms (started: 2023-01-30 17:58:42 +08:00)


In [30]:
visualizations.plot_validator_checkpoint_costs(
    df_2b,
    subplot_titles=['Multi Staking', 'Single Staking']
)

time: 231 ms (started: 2023-01-30 17:58:42 +08:00)


In [31]:
simulation_3 = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])

simulation_3.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 5,
    "PRIVATE_CHAINS_CNT": 5,
})

normal_inflationary_rate_per_year = simulation_3.model.params['inflationary_rate_per_year'][0]
simulation_3.model.params.update({
    'inflationary_rate_per_year': [
       normal_inflationary_rate_per_year,
       normal_inflationary_rate_per_year * 0.5,
       normal_inflationary_rate_per_year * 5,
    ],
    
})

df_3, _exceptions = run(simulation_3)

2023-01-30 17:58:42,966 - root - INFO - Running experiment
2023-01-30 17:58:42,967 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-30 17:58:46,234 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-30 17:58:49,566 - root - INFO - Starting simulation 0 / run 0 / subset 2
2023-01-30 17:58:53,183 - root - INFO - Experiment complete in 10.215646028518677 seconds
2023-01-30 17:58:53,186 - root - INFO - Post-processing results
2023-01-30 17:58:53,743 - root - INFO - Post-processing complete in 0.5605640411376953 seconds
time: 10.9 s (started: 2023-01-30 17:58:42 +08:00)


In [32]:
visualizations.plot_yields_per_subset_subplots(
    df_3,
    subplot_titles=['0.01', '0.005', '0.05']
)

time: 337 ms (started: 2023-01-30 17:58:53 +08:00)


In [33]:
visualizations.plot_treasury_per_subset(
    df_3,
    scenario_names=['0.01', '0.005', '0.05']
)

time: 102 ms (started: 2023-01-30 17:58:54 +08:00)


In [34]:
visualizations.plot_yields_per_subset(df_3, scenario_names={0: '0.01', 1: '0.005', 2: '0.05'})

time: 131 ms (started: 2023-01-30 17:58:54 +08:00)


In [35]:
df_3[["polygn_supply"]]

,polygn_supply
1,1.000185e+10
2,1.000369e+10
3,1.000553e+10
4,1.000737e+10
5,1.000920e+10
...,...
1540,1.590925e+10
1541,1.592373e+10
1542,1.593823e+10
1543,1.595274e+10


time: 18.7 ms (started: 2023-01-30 17:58:54 +08:00)
